<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/make_super_extend_dataset_iFish_for_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Create extended dataset for YOLOv5 implementation**

#Olympia
・Olympia dataset

Treated 640px: 561枚。テストに50枚を残す（test_olympia）、残りの511枚をtrain/valに回す

Control 640px: 169枚。テストに50枚を残す(test_control)、残りの119枚をtrain/valに回す

#Handai
・GO_extended_dataset

control: 1886枚。テストに50枚を残す(test_handai)、残りの1836枚をtrain/valに回す

treatable: 1879枚。テストに50枚を残す(test_handai)、残りの1829枚をtrain/valに回す


#合計：
treatable: 1829 + 119

control: 1836 + 511

testset: Olympia treated 50, control 50, Handai treated 50, control 50

##既存のYOLOでbounding boxを作成 --> classは既知のもので。Glaucomaのrepositoryを参考にしてYOLO datasetを作成。

In [ ]:
import pandas as pd
import os
import glob
import shutil
import glob
import random
from PIL import Image
import time
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2


# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ファイル数確認用
import os

def countfile(name, path):
    print(f"path: {path}")
    print(f"number: {len(os.listdir(path))}")
    print("")

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/


olympia_cont_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/control_640px"
olympia_grav_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px"
handai_cont_dir = "/content/GO_extended_dataset/Control_photo_1886mai"
handai_grav_dir = "/content/GO_extended_dataset/treatable"


countfile("control_640px_olympia", olympia_cont_dir)
countfile("treated_640px_olympia", olympia_grav_dir)

countfile("control_handai", handai_cont_dir)
countfile("treated_handai", handai_grav_dir)


#**Handai dataset, Olympia dataset --> super_extend_datase_for_YOLO**

In [ ]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)
os.makedirs(f"{dst_dir}/train_val")
os.makedirs(f"{dst_dir}/train_val/grav")
os.makedirs(f"{dst_dir}/train_val/cont")
os.makedirs(f"{dst_dir}/test")
os.makedirs(f"{dst_dir}/test/handai/grav")
os.makedirs(f"{dst_dir}/test/handai/cont")
os.makedirs(f"{dst_dir}/test/olympia/grav")
os.makedirs(f"{dst_dir}/test/olympia/cont")

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

random_state = 1

# Olympia Control images
src_cont = olympia_cont_dir
test_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
trainval_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

cont_images = os.listdir(src_cont)
random.Random(random_state).shuffle(cont_images)

test_cont_images = cont_images[:50]
trainval_cont_images = cont_images[50:]

for image in tqdm(test_cont_images, desc="Copying test cont images"):
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(test_cont, image)
    shutil.copy(src_path, dst_path)

for image in tqdm(trainval_cont_images, desc="Copying trainval cont images"):
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(trainval_cont, image)
    shutil.copy(src_path, dst_path)


# Olympia Grav images
src_grav = olympia_grav_dir
test_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
trainval_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"

grav_images = os.listdir(src_grav)
random.Random(random_state).shuffle(grav_images)

test_grav_images = grav_images[:50]
trainval_grav_images = grav_images[50:]

for image in tqdm(test_grav_images, desc="Copying test grav images"):
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(test_grav, image)
    shutil.copy(src_path, dst_path)

for image in tqdm(trainval_grav_images, desc="Copying trainval grav images"):
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(trainval_grav, image)
    shutil.copy(src_path, dst_path)





Copying trainval grav images: 100%|██████████| 511/511 [00:16<00:00, 30.59it/s]


In [ ]:
import os
from shutil import copyfile
from tqdm import tqdm

grav_dir = olympia_grav_dir
cont_dir = olympia_cont_dir
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# Check if files in grav_dir are in test_grav_dir
for filename in tqdm(os.listdir(grav_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_grav_dir, basename)):
        print("coooool!!!")
        continue
    else:
        copyfile(os.path.join(grav_dir, filename), os.path.join(train_val_grav_dir, filename))

# Check if files in cont_dir are in test_cont_dir
for filename in tqdm(os.listdir(cont_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_cont_dir, basename)):
        print("hotttttt!!!")
        continue
    else:
        copyfile(os.path.join(cont_dir, filename), os.path.join(train_val_cont_dir, filename))


In [ ]:
#####################
# rename
#指定されたフォルダ内にあるファイル名が"olympia_"から始まらない場合に、"olympia_"を先頭につけたファイル名にリネームする処理を行う。
#####################

import os

# フォルダのパスを指定
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# ファイル名が "olympia_" から始まらない場合、"olympia_" を先頭につけたファイル名にリネームする
def rename_file(path):
    for filename in os.listdir(path):
        if not filename.startswith("oly_"):
            new_filename = "oly_" + filename
            os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

# リネーム処理を実行
rename_file(test_grav_dir)
rename_file(test_cont_dir)
rename_file(train_val_grav_dir)
rename_file(train_val_cont_dir)


In [ ]:
print(len(os.listdir("/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav")))
print(len(os.listdir("/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont")))


511
119


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import os
import random
import shutil
from tqdm import tqdm

############################
# Handai GO_extended_dataset
#
# /treatable → /train_val/grav
# /control → /train_val/Control_photo_1886mai
#
############################

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/

#file_name, class, idxのりストを作成
def get_file_info(src_dir, class_id):
    file_list = os.listdir(src_dir)
    ids_list = [int(file_name.split('.')[0].split('-')[0]) for file_name in file_list]
    class_list = [class_id] * len(file_list)
    return file_list, ids_list, class_list

# Contフォルダの情報
cont_dir = '/content/GO_extended_dataset/Control_photo_1886mai'
file_list_cont, ids_list_cont, class_list_cont = get_file_info(cont_dir, 0)

# Gravフォルダの情報
grav_dir = '/content/GO_extended_dataset/treatable'
file_list_grav, ids_list_grav, class_list_grav = get_file_info(grav_dir, 1)

# 結合
file_list = file_list_cont + file_list_grav
ids_list = ids_list_cont + ids_list_grav
class_list = class_list_cont + class_list_grav

print(file_list)
print(ids_list)
print(class_list)



import random

# file_listに画像のパスが、ids_listに患者のidが、class_listにgravかcontか(0 or 1)が格納されていると仮定する
# 例: file_list = ['/path/to/image1.jpg', '/path/to/image2.jpg', ...]
#     ids_list = ['patient1', 'patient1', 'patient2', 'patient3', ...]
#     class_list = [0, 1, 0, 1, ...]

# ランダムシードを固定する
random.seed(1234)

# idごとに画像のリストをまとめる
id_to_files = {}
for i, id in enumerate(ids_list):
    if id not in id_to_files:
        id_to_files[id] = []
    id_to_files[id].append((file_list[i], class_list[i]))

# idごとに画像をtestかtrainに振り分ける
test_files = []
train_files = []
for id in id_to_files:
    files = id_to_files[id]
    random.shuffle(files)  # 同じid内の画像の順番をランダムに並べ替える
    for file, cls in files:
        if len(test_files) < 50 and cls == 0:
            test_files.append((file, cls))
        else:
            train_files.append((file, cls))

# 確認
print(len(test_files), len(train_files))  # 50 以上の場合は再実行してください。

In [ ]:
import os
import random
import shutil
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

############################
# Handai GO_extended_dataset
#
# /treatable → /train_val/grav
# /control → /train_val/Control_photo_1886mai
#
############################

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/

# Contフォルダのパス
src_dir = '/content/GO_extended_dataset/Control_photo_1886mai'
train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/cont'

# ファイルリストを取得し、IDを抽出してリストに格納
file_list = os.listdir(src_dir)
ids = [int(file_name.split('.')[0].split('-', 1)[0]) for file_name in file_list]

# IDごとにグループを作成して、group-stratifiedに分割
splitter = GroupShuffleSplit(n_splits=1, test_size=50, random_state=2)
train_val_indices, test_indices = next(splitter.split(file_list, groups=ids))

# 学習用とテスト用のフォルダにファイルをコピー
for i, file_name in tqdm(enumerate(file_list), total=len(file_list)):
    src_path = os.path.join(src_dir, file_name)
    dst_dir = test_dir if i in test_indices else train_val_dir
    dst_path = os.path.join(dst_dir, file_name)
    shutil.copy(src_path, dst_path)

# Gravフォルダのパス
src_dir = '/content/GO_extended_dataset/treatable'
train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/grav'

# ファイルリストを取得し、IDを抽出してリストに格納
file_list = os.listdir(src_dir)
ids = [int(file_name.split('.')[0].split('-', 1)[0]) for file_name in file_list]

# IDごとにグループを作成して、group-stratifiedに分割
splitter = GroupShuffleSplit(n_splits=1, test_size=15, random_state=4)
train_val_indices, test_indices = next(splitter.split(file_list, groups=ids))
print(len(test_indices))

# 学習用とテスト用のフォルダにファイルをコピー
for i, file_name in tqdm(enumerate(file_list), total=len(file_list)):
    src_path = os.path.join(src_dir, file_name)
    dst_dir = test_dir if i in test_indices else train_val_dir
    dst_path = os.path.join(dst_dir, file_name)
    shutil.copy(src_path, dst_path)

100%|██████████| 1886/1886 [00:44<00:00, 42.52it/s]


50


100%|██████████| 1879/1879 [00:40<00:00, 46.03it/s]


In [ ]:
# #（GroupShuffleSplitがgroupごとに分割するので）総数が50になるようにrandom_stateとtest_sizeを調節
# # 結果の格納用リスト
# # Gravフォルダのパス
# src_dir = '/content/GO_extended_dataset/treatable'
# train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav'
# test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/grav'

# # ファイルリストを取得し、IDを抽出してリストに格納
# file_list = os.listdir(src_dir)
# ids = [int(file_name.split('.')[0].split('-', 1)[0]) for file_name in file_list]

# results = []

# for random_state in range(10):
#     for test_size in range(1, 51):
#         splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
#         train_val_indices, test_indices = next(splitter.split(file_list, groups=ids))
#         test_set_size = len(test_indices)
#         print(f"random_state: {random_state}, test_size: {test_size}, test_set_size: {test_set_size}")
#         if test_set_size == 50:
#             print("Coooool!!")
#             results.append((random_state, test_size))

# # 結果の表示
# for result in results:
#     print(f"random_state: {result[0]}, test_size: {result[1]}")


#**ファイル数の確認**

In [ ]:
dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"
print(len(os.listdir(f"{dir}/train_val/grav")))
print(len(os.listdir(f"{dir}/train_val/cont")))
print(len(os.listdir(f"{dir}/test/olympia/grav")))
print(len(os.listdir(f"{dir}/test/olympia/cont")))
print(len(os.listdir(f"{dir}/test/handai/grav")))
print(len(os.listdir(f"{dir}/test/handai/cont")))

"""
2340
1933
50
50
50
50
"""

2340
1955
50
50
50
50


'\n2340\n1933\n50\n50\n50\n50\n'

#**既存のYOLOv5モデルを用いてバウンディングボックスを設定**

In [ ]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/labels"
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(f"{dst_dir}/grav")
os.makedirs(f"{dst_dir}/cont")

In [ ]:
# Setup YOLOv5
%cd /content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-72-g064365d Python-3.10.11 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.7/225.8 GB disk)


In [ ]:
from models.common import DetectMultiBackend
#from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
#from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox #padding

from PIL import Image
import torch
from torchvision import models, transforms
import cv2
import numpy as np

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


def interference(img, weight):
    device = 'cpu'
    device = select_device(device)
    model = DetectMultiBackend(weight, device=device, dnn=False)
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #imgsz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = cv2.imread(img) #CV2で開く
    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None,  max_det=1000)

    print(f"pred: {pred}")

    return pred

In [ ]:
def interference_thres(img, weight):
    device = 'cpu'
    device = select_device(device)
    model = DetectMultiBackend(weight, device=device, dnn=False)
    #stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    #imgsz = check_img_size([640], s=stride)  # check image size

    #class_names = {0:"cont", 1:"grav"}

    # transform = transforms.Compose([
    #             transforms.Resize(size=(480,640)),
    #             transforms.ToTensor(),
    #             # transforms.Normalize(
    #             #     mean=[0.5, 0.5, 0.5],
    #             #     std=[0.5, 0.5, 0.5]
    #             #    )
    #             ])

    img_cv2 = cv2.imread(img) #CV2で開く
    img_cv2 = letterbox(img_cv2, (640,640), stride=32, auto=False)[0] #resize, 上下padding (color 114)

    #cv2_imshow(img_cv2)

    img_cv2 = img_cv2.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_cv2 = np.ascontiguousarray(img_cv2)
    img_tensor = torch.from_numpy(img_cv2).float()

    #img_tensor = transform(img_np)
    img_tensor /= 255
    #print(img_tensor.shape)

    #print(img_tensor)
    img_tensor = torch.unsqueeze(img_tensor, 0)  # バッチ対応


    pred = model(img_tensor, visualize=False, augment=False)

    pred = non_max_suppression(pred, conf_thres=0.15, iou_thres=0.25, classes=None,  max_det=1000)

    #print(f"pred: {pred}")

    return pred

In [ ]:
YOLOv5_model_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/yolov5n_130epch.pt"
parent_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val"

for class_name in ["grav", "cont"]:
    if class_name == "cont":
        class_num = 0
    else:
        class_num = 1
    for image_path in glob.glob(f"{parent_dir}/{class_name}/*"):
        name = os.path.splitext(os.path.basename(image_path))[0]
        print(f"image_name: {name}")

        #YOLOv5によりバウンディングボックスを抜き出す
        try:
            pred = interference(image_path, YOLOv5_model_path)[0].tolist()[0]
        except: #通常の閾値設定で検出されない場合
            pred = interference_thres(image_path, YOLOv5_model_path)[0].tolist()[0]
        pred = [max(num, 0) for num in pred] #0より小さい座標は0に修正する
        pred = [min(num, 640) for num in pred] #640より小さい座標は640に修正する
        xcenter = ((pred[0]+pred[2])/2)/640
        ycenter = ((pred[1]+pred[3])/2)/640
        xwidth = (pred[2]-pred[0])/640
        ywidth = (pred[3]-pred[1])/640

        # ファイル名と作成する内容を指定
        txt_name = f"{parent_dir}/labels/{class_name}/{name}.txt"
        content = f"{class_num} {xcenter} {ycenter} {xwidth} {ywidth}"
        with open(txt_name, "w") as file:
            # テキストをファイルに書き込む
            file.write(content)
            print(f"text file successfully created🙆 {content}")
            print("")
            print("")


Fusing layers... 


pred: [tensor([[ 64.66748, 227.64108, 514.43115, 399.88657,   0.88143,   0.00000]])]
text file successfully created🙆 0 0.4524208068847656 0.4902559757232666 0.7027557373046875 0.2691335678100586


image_name: 9808-20191114-22-094457_e056ae61aa8b734ef2992b52c8fe38add031cae5e048f4d91553a02c64b6c645


Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.11 torch-2.0.1+cu118 CPU

Fusing layers... 


pred: [tensor([[-5.88837e-01,  1.90427e+02,  6.37211e+02,  4.48986e+02,  8.17542e-01,  0.00000e+00]])]
text file successfully created🙆 0 0.49782142639160154 0.4995415687561035 0.9956428527832031 0.40399904251098634


image_name: 8027


Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.11 torch-2.0.1+cu118 CPU

Fusing layers... 


...

pred: [tensor([[  8.33530, 164.84067, 638.79395, 450.50369,   0.81064,   0.00000]])]
text file successfully created🙆 0 0.5055697202682495 0.4807377815246582 0.9850916385650634 0.4463484764099121


image_name: 4163


Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.11 torch-2.0.1+cu118 CPU

Fusing layers... 


pred: [tensor([[ 12.76440, 178.12775, 633.55420, 446.41986,   0.86880,   0.00000]])]
text file successfully created🙆 0 0.504936408996582 0.48792781829833987 0.9699840545654297 0.419206428527832


image_name: 5513


Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.11 torch-2.0.1+cu118 CPU

Fusing layers... 


pred: [tensor([[  5.46478, 185.79282, 633.87360, 426.82767,   0.85804,   0.00000]])]
text file successfully created🙆 0 0.4994831085205078 0.4786097526550293 0.9818887710571289 0.3766169548034668


image_name: 4183


In [ ]:
dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val"
print(len(os.listdir(f"{dir}/grav")))
print(len(os.listdir(f"{dir}/cont")))
print(len(os.listdir(f"{dir}/labels/grav")))
print(len(os.listdir(f"{dir}/labels/cont")))

"""
2340
1933
2340
1933
"""

#**YOLOv5用トレーニング用フォルダ作成**

https://github.com/ykitaguchi77/GravCont_classification_colab/blob/master/Extend_dataset_YOLOv5%EF%BC%A0.ipynb

https://book.st-hakky.com/docs/object-detection-yolov5-tutorial/

In [ ]:
"""

---------------super_extend_for_YOLO_training --------train------labels
                                              |           |--images
                                              |
                                              |------valid ----labels
                                              |           |--images
                                              |
                                              |--dataset.yaml
                                              |--yolov5
"""

In [ ]:
dst_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training"
orig_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val"


In [ ]:
if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path)
os.makedirs(f"{dst_path}/train/images")
os.makedirs(f"{dst_path}/train/labels")
os.makedirs(f"{dst_path}/valid/images")
os.makedirs(f"{dst_path}/valid/labels")

In [ ]:
%cd $dst_path

In [ ]:
%%writefile dataset.yaml
# path
train: /content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/train/images
val: /content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/valid/images

# num of classes
nc: 2

#class names
names: ['cont', 'grav'] # class名を定義

In [ ]:
# create dataset_list

import numpy as np
from sklearn.model_selection import train_test_split
import glob
import os
import shutil

dataset_dir =  "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/train_val"


# def split_dataset(dataset_dir):
#     img_list = glob.glob(f"{dataset_dir}/images/*")
#     img_train, img_test = train_test_split(img_list, test_size=0.3, random_state=0)

#     # img_train, img_testに名前が一致するtxtファイルを抜き出す
#     label_train = [f"{dataset_dir}/labels/{os.path.basename(i).split('.')[0]}.txt" for i in img_train]
#     label_test = [f"{dataset_dir}/labels/{os.path.basename(i).split('.')[0]}.txt" for i in img_test]

#     print(f"train: {len(label_train)},test: {len(label_test)}")

#     return img_train, img_test, label_train, label_test

def make_path_list(dir, class_name):
    image_list =  [file for file in glob.glob(f"{dir}/{class_name}/*") if os.path.isfile(file) == True ]
    label_list =  [f"{dir}/{class_name}/labels/{os.path.basename(i).split('.')[0]}.txt" for i in image_list]

    id_list = [os.path.basename(i).split("-")[0].split(".")[0] for i in image_list]

    index = {}
    id_idx = []
    for item in id_list:
        if item in index:
            id_idx.append(index[item])
        else:
            index[item] = len(index) + 1
            id_idx.append(index[item])
    id_idx = [int(i) for i in id_idx]

    print(f"image_list: {image_list}")
    print(f"label_list: {label_list}")

    return image_list, label_list, id_idx

grav_image_list, grav_label_list, grav_id_idx = make_path_list(orig_path, "grav")
cont_image_list, cont_label_list, cont_id_idx = make_path_list(orig_path, "cont")

print("image")
print(f"grav: {len(grav_image_list)}")
print(f"cont: {len(cont_image_list)}")
print("label")
print(f"grav: {len(grav_label_list)}")
print(f"cont: {len(cont_label_list)}")

In [ ]:
# Split dataset using GroupShuffleSplit

import os
from tqdm import tqdm
import random
import shutil
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

############################
# 以下をGroupShuffleSplitで分ける
# super_extend_dataset_for_YOLO/train_val/grav
# → /super_extend_for_YOLO_training/train/images
# → /super_extend_for_YOLO_training/vallid/images
#
# super_extend_dataset_for_YOLO/train_val/cont
# → /super_extend_for_YOLO_training/train/images
# → /super_extend_for_YOLO_training/vallid/images
#
# 以下はbasenameが同じものを移動するのみとする
# super_extend_dataset_for_YOLO/train_val/label/grav
# → /super_extend_for_YOLO_training/train/labels
# → /super_extend_for_YOLO_training/vallid/labels
#
# super_extend_dataset_for_YOLO/train_val/label/grav
# → /super_extend_for_YOLO_training/train/labels
# → /super_extend_for_YOLO_training/vallid/labels
############################

dst_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training"
orig_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val"

for class_name in ["cont", "grav"]:
    # フォルダのパス
    src_dir = f"{orig_folder}/{class_name}"
    train_dir = f"{dst_folder}/train/images"
    valid_dir = f"{dst_folder}/valid/images"

    # ファイルリストを取得し、IDを抽出してリストに格納
    file_list = os.listdir(src_dir)
    ids = [str(file_name.split('.')[0].split('-', 1)[0]) for file_name in file_list]

    # IDごとにグループを作成して、group-stratifiedに分割
    splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=2)
    train_indices, valid_indices = next(splitter.split(file_list, groups=ids))

    # 学習用とテスト用のフォルダにファイルをコピー
    for i, file_name in tqdm(enumerate(file_list), total=len(file_list)):
        src_path = os.path.join(src_dir, file_name)
        dst_dir = valid_dir if i in valid_indices else train_dir
        dst_path = os.path.join(dst_dir, file_name)
        shutil.copy(src_path, dst_path)

        label_file_name = file_name.rsplit(".", 1)[0] + ".txt" #同じ名前のラベルファイルも移動
        #print(f"label_file_name: {label_file_name}")
        label_dst_dir = os.path.dirname(dst_path).replace("images", "labels")
        #print(f"label_dst_dir: {label_dst_dir}")

        label_src_path = os.path.join(orig_folder, "labels", class_name, label_file_name)
        label_dst_path = os.path.join(label_dst_dir, label_file_name)
        shutil.copy(label_src_path, label_dst_path)

In [ ]:
print(f"train_images: {len(os.listdir('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/train/images'))}")
print(f"val_images: {len(os.listdir('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/valid/images'))}")
print(f"train_labels: {len(os.listdir('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/train/labels'))}")
print(f"val_labels: {len(os.listdir('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/valid/labels'))}")


"""
train_images: 3480
val_images: 792
train_labels: 3480
val_labels: 792
"""


In [ ]:
dst_dir

'/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_for_YOLO_training/train/images'